<a href="https://colab.research.google.com/github/JCaballerot/Deep_learning_program/blob/main/Deep_learning_program/Deep_learning_program/Modulo_IV/Lab_Collaborative_Filtering_con_RBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align="center"><font size="5"> SISTEMAS DE RECOMENDACION USANDO RESTRICTED BOLTZMANN MACHINE</font></h1>

Bienvenido al laboratorio <b>Sistema de recomendaciones usando restricted Boltzmann machine's</b>. En este cuaderno, estudiamos y repasamos el uso de una máquina restringida de Boltzmann (RBM) en un sistema de recomendación basado en colaborative filtering. Este sistema es un algoritmo que recomienda elementos al tratar de encontrar usuarios que sean similares entre sí en función de las calificaciones de sus elementos. Al final de este laboratorio, debería tener una comprensión más profunda de cómo se aplican las máquinas restringidas de Boltzmann y cómo crear una con TensorFlow.

<h2>Tabla de Contenidos</h2>

<ol>
    <li><a href="#ref1">Obtener la data</a></li>
    <li><a href="#ref2">Carga de la data</a></li>
    <li><a href="#ref3">El modelo de Restricted Boltzmann machine</a></li>
    <li><a href="#ref4">Configuración de los parámetros del modelo</a></li>
    <li><a href="#ref5">Recomendación</a></li>
</ol>
<br>
<br>
<hr>

<a id="ref1"></a>
<h2>Obtener la Data</h2>

Para empezar, necesitamos descargar los datos que vamos a utilizar para nuestro sistema. Los conjuntos de datos que vamos a usar fueron adquiridos por <a href="http://grouplens.org/datasets/movielens/"> GroupLens </a> y contienen películas, usuarios y calificaciones de películas de estos usuarios.

Después de descargar los datos, extraeremos los conjuntos de datos a un directorio que sea de fácil acceso.

In [4]:
%%capture
!pip3 install tensorflow

In [7]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [8]:
%%capture
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip

Con los conjuntos de datos en su lugar, ahora importemos las bibliotecas necesarias. Usaremos <a href="https://www.tensorflow.org/"> Tensorflow </a> y <a href="http://www.numpy.org/"> Numpy </a> juntos para modelar e inicializar nuestra máquina restringida de Boltzmann y <a href="http://pandas.pydata.org/pandas-docs/stable/"> Pandas </a> para manipular nuestros conjuntos de datos. Para importar estas bibliotecas, ejecute la celda de código a continuación.

In [9]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

<hr>

<a id="ref2"> </a>
<h2> Carga de la data</h2>

Comencemos cargando nuestros datos con Pandas. Los archivos .dat que contienen nuestros datos son similares a los archivos CSV, pero en lugar de usar el carácter ',' (coma) para separar entradas, usa caracteres '::' (dos dos puntos). Para que Pandas sepa que debe separar puntos de datos en cada '::', tenemos que especificar el parámetro <code> sep = '::' </code> al llamar a la función.

Además, también le pasamos el parámetro <code> header = None </code> debido al hecho de que nuestros archivos no contienen ningún encabezado.

Comencemos con el archivo movies.dat y echemos un vistazo a su estructura:

In [ ]:
#Cargando nuestro dataset de peliculas
movies_df = pd.read_csv('movies.csv')
movies_df.head()

Podemos hacer lo mismo con el archivo ratings.dat:

In [ ]:
#Cargando los puntajes de las películas
ratings_df = pd.read_csv('ratings.csv')
ratings_df.head()

Por lo tanto, nuestra variable <b>movies_df</b> contiene un marco de datos que almacena el número de identificación, el título y los géneros únicos de una película, mientras que nuestra variable <b>ratings_df</b> almacena un número de identificación de usuario único, la identificación de una película que el usuario ha visto, la calificación del usuario para dicha película y cuándo el usuario calificó esa película.

Cambiemos el nombre de las columnas en estos marcos de datos para que podamos transmitir mejor sus datos de manera más intuitiva:

In [ ]:
movies_df.columns = ['MovieID', 'Title', 'Genres']
movies_df.head()

In [ ]:
movies_df[movies_df.MovieID == 48516]

y ratings_df:

In [ ]:
ratings_df.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
ratings_df.head()

<hr>

<a id="ref3"></a>
<h2>El modelo de Restricted Boltzmann Machine's</h2>

<img src="https://ibm.box.com/shared/static/o049tx0dsllpbj3b546vuba25qqlzelq.png"  width="300">
<br>
El modelo de la máquina restringida de Boltzmann tiene dos capas de neuronas, una de las cuales es lo que llamamos una capa de entrada visible y la otra se llama capa oculta. La capa oculta se utiliza para aprender características de la información que se alimenta a través de la capa de entrada. Para nuestro modelo, la entrada contendrá X neuronas, donde X es la cantidad de películas en nuestro conjunto de datos. Cada una de estas neuronas poseerá un valor de calificación normalizado que varía de 0 a 1, donde 0 significa que un usuario no ha visto esa película y cuanto más cerca está el valor de 1, más le gusta al usuario la película que representa la neurona. Estos valores normalizados, por supuesto, se extraerán y normalizarán del conjunto de datos de calificaciones.

Después de pasar la entrada, entrenamos el RBM en ella y hacemos que la capa oculta aprenda sus características. Estas funciones son las que usamos para reconstruir la entrada, que en nuestro caso, predecirá las calificaciones de las películas que el usuario no ha visto, ¡que es exactamente lo que podemos usar para recomendar películas!

Ahora comenzaremos a formatear nuestro conjunto de datos para seguir la entrada esperada del modelo.

<h3>Formateando la Data</h3>

Primero veamos cuántas películas tenemos y veamos si los ID de la película se corresponden con ese valor:

In [ ]:
movies_df.shape

In [ ]:
ratings_df = ratings_df.drop('Timestamp', 1)

In [ ]:
ratings_df.shape

In [ ]:
ratings_df = ratings_df.loc[ratings_df.UserID <= 1000]
ratings_df.head()
ratings_df.shape

In [ ]:
ratings_df.shape

Ahora, podemos comenzar a formatear los datos en entrada para el RBM. Vamos a almacenar las calificaciones de usuarios normalizadas en una matriz de calificación de usuarios llamada trX y normalizar los valores.

In [ ]:
ratings_df.head()

In [ ]:
user_rating_df = ratings_df.pivot(index='UserID', columns='MovieID', values='Rating')
user_rating_df.head()

Normalizamos la data:

In [ ]:
norm_user_rating_df = user_rating_df.fillna(0)/ 5.0
trX = norm_user_rating_df.values
trX[0:5]

<hr>

<a id="ref4"></a>
<h2>Seteando los parámetros del modelo</h2>

A continuación, comencemos a construir nuestro RBM con TensorFlow. Comenzaremos determinando primero la cantidad de neuronas en las capas ocultas y luego creando variables de marcador de posición para almacenar nuestros sesgos de capa visible, sesgos de capa oculta y pesos que conectan la capa oculta con la capa visible. Estableceremos arbitrariamente el número de neuronas en las capas ocultas en 20. Puede establecer libremente este valor en cualquier número que desee, ya que cada neurona en la capa oculta terminará aprendiendo una característica.

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Input
import numpy as np


In [47]:
# Definir el número de unidades en la capa visible y oculta
visible_units = len(user_rating_df.columns)  # Reemplaza esto con el número real de características de tu entrada
hidden_units = 200

In [48]:
# Crear el modelo
model = Sequential()
model.add(Dense(hidden_units, input_dim = visible_units, activation='sigmoid', name='hidden_layer'))
model.add(Dense(visible_units, activation='relu', name='output_layer'))


In [ ]:
# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Mostrar la estructura del modelo
model.summary()

In [50]:
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

In [51]:
# Configurar Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience = 10, verbose=1, restore_best_weights=True)


In [53]:
# Establecer el learning rate específico para el optimizador
learning_rate = 0.05
optimizer = Adam(learning_rate = learning_rate)

In [ ]:
epochs = 100
batchsize = 200

history = model.fit(trX, trX, validation_split = 0.2, epochs = epochs, batch_size = batchsize, callbacks=[early_stopping])

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8, 4))
plt.grid(True)
plt.gca().set_ylim(0, 0.007) # set the vertical range to [0-1] plt.show()

<hr>

<a id="ref5"></a>
<h2>Recomendación</h2>

Ahora podemos predecir películas que podrían gustarle a un usuario seleccionado arbitrariamente. Esto se puede lograr introduciendo las preferencias de películas vistas del usuario en el RBM y luego reconstruyendo la entrada. Los valores que nos da el RBM intentarán estimar las preferencias del usuario por las películas que no ha visto en función de las preferencias de los usuarios en los que se entrenó el RBM.

Primero seleccionemos un <b>User ID</b> de nuestra data:

In [57]:
mock_user_id = 2

In [ ]:
#Seleccionando al usuario input
inputUser = trX[mock_user_id-1].reshape(1, -1)
inputUser[0:5]

In [ ]:
rec = model.predict(inputUser)
print(rec)

Luego, podemos enumerar las 20 películas más recomendadas para nuestro usuario simulado clasificándolas por sus puntajes dados por nuestro modelo.

In [ ]:
scored_movies_df_mock = movies_df[movies_df['MovieID'].isin(user_rating_df.columns)]
scored_movies_df_mock = scored_movies_df_mock.assign(RecommendationScore = rec[0])
scored_movies_df_mock.sort_values(["RecommendationScore"], ascending=False).head(20)

Entonces, ¿cómo recomendar las películas que el usuario aún no ha visto?

Ahora, podemos encontrar todas las películas que nuestro usuario ha visto antes:

In [ ]:
movies_df_mock = ratings_df[ratings_df['UserID'] == mock_user_id]
movies_df_mock.head()

En la siguiente celda, cruzamos todas las películas que nuestro usuario ha visto con los puntajes predichos basados en sus datos históricos:

In [63]:
#Cruzandi movies_df con ratings_df por MovieID
merged_df_mock = scored_movies_df_mock.merge(movies_df_mock, on='MovieID', how='outer')

ordenemos y echemos un vistazo a las primeras 20 filas:

In [ ]:
merged_df_mock.sort_values(["Rating"], ascending=False).head(20)

Como puede ver, hay algunas películas que el usuario aún no ha visto y tiene una puntuación alta según nuestro modelo. Entonces, podemos recomendarlos al usuario.

Puede intentar cambiar los parámetros en el código: agregar más unidades a la capa oculta, cambiar las funciones de pérdida o tal vez algo más para ver si cambia algo. ¿El modelo funciona mejor? ¿Se tarda más en calcular?


<hr>

### Gracias por completar este laboratorio!